In [1]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging

#utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
from dense_correspondence.dataset.dynamic_spartan_dataset import DynamicSpartanDataset
logging.basicConfig(level=logging.INFO)

from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation


/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
W0731 17:45:45.290409 140544686176000 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


setting CUDA_VISIBLE_DEVICES =  0,


## Load the configuration for training

In [2]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence',
                                       'dataset', 'composite',
                                       'dynamic_sugar_move_to_box_0710.yaml')
config = utils.getDictFromYamlFilename(config_filename)
config["provide_gt_depths"] = True

train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')

train_config = utils.getDictFromYamlFilename(train_config_file)
dataset = DynamicSpartanDataset(config=config)
dataset._augment_data = False

logging_dir = "trained_models/imitation"
num_iterations = 100000
d = 8 # the descriptor dimension
name = "sugar_logitdetection_0710_%d-lrs0.01-learnedkernel-maxover" %(d)
train_config["training"]["logging_dir_name"] = name
train_config["training"]["logging_dir"] = logging_dir
train_config["dense_correspondence_network"]["descriptor_dimension"] = d
train_config["training"]["num_iterations"] = num_iterations
train_config["dense_correspondence_network"]["compute_best_match_with"] = "learned_kernel"
train_config["dense_correspondence_network"]["normalize"] = False
train_config["training"]["save_rate"] = 1000
train_config["training"]["steps_between_learning_rate_decay"] = 1000

TRAIN = True
EVALUATE = False

imitation/move_to_box_careful
Using SpartanDataset:
   - in train mode
   - number of scenes 200
   - total images:     0


## Train the network

This should take about ~12-15 minutes with a GTX 1080 Ti

In [ ]:
# All of the saved data for this network will be located in the
# code/data_volume/pdc/trained_models/tutorials/caterpillar_3 folder

if TRAIN:
    print "training descriptor of dimension %d" %(d)
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    #train.run()
    #train.run_from_pretrained("imitation/sugar_contrastive_0710_wblind_3")
    #train.run(use_spatial_softmax_only=True)
    #train.run_from_pretrained("imitation/sugar_spatial_3", use_spatial_softmax_only=True)
    # NEW
    train.run(use_spatial_softmax_only=True, train_logit_detection=True)
    print "finished training descriptor of dimension %d" %(d)


training descriptor of dimension 8
using SINGLE_OBJECT_WITHIN_SCENE
logging_dir: /home/peteflo/data/pdc/trained_models/imitation/sugar_logitdetection_0710_8-lrs0.01-learnedkernel-maxover
single iteration took 0.288 seconds
single iteration took 0.152 seconds
single iteration took 0.154 seconds
single iteration took 0.158 seconds
single iteration took 0.157 seconds
single iteration took 0.156 seconds
single iteration took 0.154 seconds
single iteration took 0.155 seconds
single iteration took 0.156 seconds
single iteration took 0.156 seconds
single iteration took 0.152 seconds
single iteration took 0.160 seconds
single iteration took 0.160 seconds
single iteration took 0.154 seconds
single iteration took 0.152 seconds
single iteration took 0.159 seconds
single iteration took 0.150 seconds
single iteration took 0.151 seconds
single iteration took 0.157 seconds
single iteration took 0.155 seconds
single iteration took 0.159 seconds
single iteration took 0.157 seconds
single iteration took

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print count_parameters(train._dcn)

## Evaluate the network quantitatively

This should take ~5 minutes.

In [ ]:
os.listdir("/home/peteflo/data")

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
print model_folder
model_folder = "/home/peteflo/data/imitation/trained_models/tutorials/first_real_3"

if True:
    DCE = DenseCorrespondenceEvaluation
    num_image_pairs = 100
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)      

See `evaluation_quantitative_tutorial.ipynb` for a better place to display the plots.